<a href="https://colab.research.google.com/github/jacksontenorio8/FakeNews/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('portuguese'))

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [4]:
df = pd.read_csv('/content/drive/MyDrive/fakenews/true.csv', encoding='utf-8')

# Nova secção

In [5]:
df.head()

,title,text,origin,url,label,publisher_name,publisher_site,date
0,Jovem Pan: o que é falso e o que é verdadeiro ...,As propostas de manter zerados impostos federa...,Jair Bolsonaro,https://lupa.uol.com.br/jornalismo/2022/09/06/...,0,Lupa - UOL,lupa.uol.com.br,NaN
1,Jovem Pan: o que é falso e o que é verdadeiro ...,Estatais dão lucro de mais de R$ 100 bilhões e...,Jair Bolsonaro,https://lupa.uol.com.br/jornalismo/2022/09/06/...,0,Lupa - UOL,lupa.uol.com.br,NaN
2,De cargos secretos a segurança: veja erros e a...,A água foi privatizada. A Cedae não acabou. Es...,Marcelo Freixo (PSB),https://lupa.uol.com.br/jornalismo/2022/09/01/...,0,Lupa - UOL,lupa.uol.com.br,NaN
3,De cargos secretos a segurança: veja erros e a...,São quase 3 milhões de pessoas passando fome [...,Marcelo Freixo (PSB),https://lupa.uol.com.br/jornalismo/2022/09/01/...,0,Lupa - UOL,lupa.uol.com.br,NaN
4,De cargos secretos a segurança: veja erros e a...,[Cláudio Castro] tem pouco mais de 20% de ótim...,Marcelo Freixo (PSB),https://lupa.uol.com.br/jornalismo/2022/09/01/...,0,Lupa - UOL,lupa.uol.com.br,NaN


In [6]:
df = pd.read_csv('/content/drive/MyDrive/fakenews/fakes.csv',encoding='utf-8')

In [7]:
df.isnull().sum()

title                0
text                 0
origin            4644
url                  0
label                0
publisher_name     363
publisher_site       0
date              3036
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.isnull().sum()

title             0
text              0
origin            0
url               0
label             0
publisher_name    0
publisher_site    0
date              0
dtype: int64

In [10]:
df['all'] = df['title'] + ' ' + df['origin'] + ' ' + df['text'] 

In [11]:
pd.options.display.max_colwidth = 150

In [12]:
print(df['all'])

1        Documentos confirmam uso de dinheiro vivo na compra de 51 imóveis da família Bolsonaro Cristina Graeml “Não foi dinheiro vivo. Moeda corrente, que...
4        É falso que órgão de saúde dos EUA passou a recomendar ivermectina para Covid-19 Post no WhatsApp De repente, a Ivermectina aparece no site do NIH...
5        É falso que arma usada em atentado contra Cristina Kirchner era uma pistola d'água Publicação no Facebook Mais uma palhaçada da Cristina Kirchner ...
6        Foto antiga do Burj Khalifa com as cores do Brasil viralizou nas redes Publicação no Facebook O edifício mais alto do mundo, o Burj Khalifa, em Du...
11       É antigo vídeo em que paraquedistas comentam sobre a expectativa para os atos de 7 de setembro Publicação no WhatsApp Clima ontem a noite em Brasí...
                                                                                 ...                                                                          
20473    Paulo Guedes gastou mesmo R$ 120 mil 

In [13]:
porter_stemmer = PorterStemmer()


def stemming_tokenizer(df):

    words= re.sub(r"[^A-Za-z]", " ",df).lower().split()
    words= [porter_stemmer.stem(word) for word in words if not words in stopwords.words('portuguese')]
    words= ' '.join(words)
    return words

In [14]:
df['features'] = df['all'].apply(stemming_tokenizer)

In [15]:
print(df['features'])

1        documento confirmam uso de dinheiro vivo na compra de im vei da fam lia bolsonaro cristina graeml n o foi dinheiro vivo moeda corrent quer dizer o...
4        falso que rg o de sa de do eua passou a recomendar ivermectina para covid post no whatsapp de repent a ivermectina aparec no site do nih para trat...
5        falso que arma usada em atentado contra cristina kirchner era uma pistola d gua publica o no facebook mai uma palha ada da cristina kirchner comun...
6        foto antiga do burj khalifa com as core do brasil viraliz na rede publica o no facebook o edif cio mai alto do mundo o burj khalifa em dubai est i...
11       antigo v deo em que paraquedista comentam sobr a expectativa para os ato de de setembro publica o no whatsapp clima ontem a noit em bra lia aguard...
                                                                                 ...                                                                          
20473             paulo gued gastou mesmo r mi

In [22]:
X = df['features'].values
Y = df['label'].values

In [24]:
Y

array([1, 1, 1, ..., 1, 1, 1])

In [25]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

TfidfVectorizer()

In [26]:
X = vectorizer.transform(X)

In [16]:
df.head()

,title,text,origin,url,label,publisher_name,publisher_site,date,all,features
1,Documentos confirmam uso de dinheiro vivo na compra de 51 imóveis da família Bolsonaro,"“Não foi dinheiro vivo. Moeda corrente, quer dizer o dinheiro do país. Então, se um imóvel foi comprado em moeda corrente, significa que ele foi c...",Cristina Graeml,https://lupa.uol.com.br/jornalismo/2022/09/09/dinheiro-vivo-uol,1,Lupa - UOL,lupa.uol.com.br,2022-09-06T00:00:00Z,"Documentos confirmam uso de dinheiro vivo na compra de 51 imóveis da família Bolsonaro Cristina Graeml “Não foi dinheiro vivo. Moeda corrente, que...",documento confirmam uso de dinheiro vivo na compra de im vei da fam lia bolsonaro cristina graeml n o foi dinheiro vivo moeda corrent quer dizer o...
4,É falso que órgão de saúde dos EUA passou a recomendar ivermectina para Covid-19,"De repente, a Ivermectina aparece no site do NIH para tratar o Covid. Depois de anos de negação, bloqueio, interferência, difamação, arruinando co...",Post no WhatsApp,https://lupa.uol.com.br/jornalismo/2022/09/08/orgao-saude-eua-ivermectina-covid-19,1,Lupa - UOL,lupa.uol.com.br,2022-09-08T00:00:00Z,"É falso que órgão de saúde dos EUA passou a recomendar ivermectina para Covid-19 Post no WhatsApp De repente, a Ivermectina aparece no site do NIH...",falso que rg o de sa de do eua passou a recomendar ivermectina para covid post no whatsapp de repent a ivermectina aparec no site do nih para trat...
5,É falso que arma usada em atentado contra Cristina Kirchner era uma pistola d'água,"Mais uma palhaçada da Cristina Kirchner Comunista, tudo foi uma armação não tinha bala na pistola 🔫 tinha água e a mídia fazendo o trabalho sujo d...",Publicação no Facebook,https://lupa.uol.com.br/jornalismo/2022/09/07/cristina-kirchner-pistola-agua,1,Lupa - UOL,lupa.uol.com.br,2022-09-02T00:00:00Z,É falso que arma usada em atentado contra Cristina Kirchner era uma pistola d'água Publicação no Facebook Mais uma palhaçada da Cristina Kirchner ...,falso que arma usada em atentado contra cristina kirchner era uma pistola d gua publica o no facebook mai uma palha ada da cristina kirchner comun...
6,Foto antiga do Burj Khalifa com as cores do Brasil viralizou nas redes,"O edifício mais alto do mundo, o Burj Khalifa, em Dubai, está iluminado com as cores da bandeira brasileira por conta do Dia da Independência”",Publicação no Facebook,https://lupa.uol.com.br/jornalismo/2022/09/07/burj-khalifa-7-setembro,1,Lupa - UOL,lupa.uol.com.br,2022-09-06T00:00:00Z,"Foto antiga do Burj Khalifa com as cores do Brasil viralizou nas redes Publicação no Facebook O edifício mais alto do mundo, o Burj Khalifa, em Du...",foto antiga do burj khalifa com as core do brasil viraliz na rede publica o no facebook o edif cio mai alto do mundo o burj khalifa em dubai est i...
11,É antigo vídeo em que paraquedistas comentam sobre a expectativa para os atos de 7 de setembro,"Clima ontem a noite em Brasília, aguardando o 7 de setembro",Publicação no WhatsApp,https://lupa.uol.com.br/jornalismo/2022/09/07/paraquedistas-atos-7-setembro,1,Lupa - UOL,lupa.uol.com.br,2022-09-07T00:00:00Z,É antigo vídeo em que paraquedistas comentam sobre a expectativa para os atos de 7 de setembro Publicação no WhatsApp Clima ontem a noite em Brasí...,antigo v deo em que paraquedista comentam sobr a expectativa para os ato de de setembro publica o no whatsapp clima ontem a noit em bra lia aguard...


In [17]:
df.isnull().sum()

title             0
text              0
origin            0
url               0
label             0
publisher_name    0
publisher_site    0
date              0
all               0
features          0
dtype: int64

In [18]:
df.dropna(inplace=True)

In [19]:
df.isnull().sum()

title             0
text              0
origin            0
url               0
label             0
publisher_name    0
publisher_site    0
date              0
all               0
features          0
dtype: int64

In [20]:
df['all'] = df['title'] + ' ' + df['origin'] + ' ' + df['text']

In [21]:
pd.options.display.max_colwidth = 150
print(df['all'])

1        Documentos confirmam uso de dinheiro vivo na compra de 51 imóveis da família Bolsonaro Cristina Graeml “Não foi dinheiro vivo. Moeda corrente, que...
4        É falso que órgão de saúde dos EUA passou a recomendar ivermectina para Covid-19 Post no WhatsApp De repente, a Ivermectina aparece no site do NIH...
5        É falso que arma usada em atentado contra Cristina Kirchner era uma pistola d'água Publicação no Facebook Mais uma palhaçada da Cristina Kirchner ...
6        Foto antiga do Burj Khalifa com as cores do Brasil viralizou nas redes Publicação no Facebook O edifício mais alto do mundo, o Burj Khalifa, em Du...
11       É antigo vídeo em que paraquedistas comentam sobre a expectativa para os atos de 7 de setembro Publicação no WhatsApp Clima ontem a noite em Brasí...
                                                                                 ...                                                                          
20473    Paulo Guedes gastou mesmo R$ 120 mil 